In [1]:
# Some part of the code was referenced from below.
# https://github.com/pytorch/examples/tree/master/word_language_model
import torch
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm_
from data_utils import Dictionary, Corpus

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper-parameters
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 5
num_samples = 1000  # number of words to be sampled
batch_size = 20
seq_length = 30
learning_rate = 0.002

In [4]:
# Load "Penn Treebank" dataset
corpus = Corpus()
ids = corpus.get_data('data/train.txt', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

In [7]:
# RNN based language model
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x, h):
        # Embed word ids to vectors
        x = self.embed(x)
        
        # Forward propagate LSTM
        out, (h, c) = self.lstm(x, h)
        
        # Reshape output to (batch_size*sequence_length, hidden_size)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        
        # Decode hidden states of all time steps
        out = self.linear(out)
        return out, (h, c)

In [8]:
model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Truncated backpropagation
def detach(states):
    return [state.detach() for state in states]

In [11]:
# Train the model
for epoch in range(num_epochs):
    # Set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
              torch.zeros(num_layers, batch_size, hidden_size).to(device))
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        
        # Forward pass
        states = detach(states)
        outputs, states = model(inputs, states)
        loss = criterion(outputs, targets.reshape(-1))
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        
        step = (i+1) // seq_length
        if step % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{step}/{num_batches}], Loss: {loss.item():.4f}, Perplexity: {np.exp(loss.item()):5.2f}')

Epoch [1/5], Step [0/1549], Loss: 9.2102, Perplexity: 9998.21
Epoch [1/5], Step [100/1549], Loss: 6.0192, Perplexity: 411.26
Epoch [1/5], Step [200/1549], Loss: 5.9327, Perplexity: 377.17
Epoch [1/5], Step [300/1549], Loss: 5.7564, Perplexity: 316.21
Epoch [1/5], Step [400/1549], Loss: 5.6436, Perplexity: 282.47
Epoch [1/5], Step [500/1549], Loss: 5.1256, Perplexity: 168.28
Epoch [1/5], Step [600/1549], Loss: 5.2027, Perplexity: 181.77
Epoch [1/5], Step [700/1549], Loss: 5.3540, Perplexity: 211.44
Epoch [1/5], Step [800/1549], Loss: 5.1484, Perplexity: 172.15
Epoch [1/5], Step [900/1549], Loss: 5.1248, Perplexity: 168.13
Epoch [1/5], Step [1000/1549], Loss: 5.1213, Perplexity: 167.55
Epoch [1/5], Step [1100/1549], Loss: 5.3443, Perplexity: 209.41
Epoch [1/5], Step [1200/1549], Loss: 5.1668, Perplexity: 175.35
Epoch [1/5], Step [1300/1549], Loss: 5.1043, Perplexity: 164.73
Epoch [1/5], Step [1400/1549], Loss: 4.8183, Perplexity: 123.76
Epoch [1/5], Step [1500/1549], Loss: 5.1302, Perple

In [13]:
# Test the model
with torch.no_grad():
    with open('sample.txt', 'w') as f:
        # Set initial hidden and cell states
        state = (torch.zeros(num_layers, 1, hidden_size).to(device),
                 torch.zeros(num_layers, 1, hidden_size).to(device))
        
        # Select on word id randomly
        prob = torch.ones(vocab_size)
        input = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device)
        
        for i in range(num_samples):
            # Forward propagate RNN
            output, state = model(input, state)
            
            # Sample a word id
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()
            
            # Fill input with sampled word id for the next time step
            input.fill_(word_id)
            
            # File write
            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)
            
            if (i+1) % 100 == 0:
                print(f'Sampled [{i+1}/{num_samples}] words and save to {"sample.txt"}')

Sampled [100/1000] words and save to sample.txt
Sampled [200/1000] words and save to sample.txt
Sampled [300/1000] words and save to sample.txt
Sampled [400/1000] words and save to sample.txt
Sampled [500/1000] words and save to sample.txt
Sampled [600/1000] words and save to sample.txt
Sampled [700/1000] words and save to sample.txt
Sampled [800/1000] words and save to sample.txt
Sampled [900/1000] words and save to sample.txt
Sampled [1000/1000] words and save to sample.txt


In [14]:
# Save the model checkpoints
torch.save(model.state_dict(), 'model.ckpt')